# Install needed libraries

In [1]:
!pip install sentence_transformers
!pip install torch
!pip install proto-plus==1.24.0.dev1 
!pip install qdrant-client
!pip install huggingface_hub
!pip install google-generativeai
!pip install tranformers
!pip install --upgrade huggingface_hub
!pip install openai pydantic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 2.1 MB/s eta 0:00:00
  Attempting uninstall: proto-plus
    Found existing installation: proto-plus 1.25.0
    Uninstalling proto-plus-1.25.0:
      Successfully uninstalled proto-plus-1.25.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-cloud-bigtable 2.27.0 requires google-api-core[grpc]<3.0.0dev,>=2.16.0, but you have google-api-core 1.34.1 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 306.6/306.6 kB 6.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 42.9 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 63.2 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.7/319.7 kB 21.3 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling prot

# Import needed libraries

In [2]:
from kaggle_secrets import UserSecretsClient
import random
import torch
import numpy as np
from huggingface_hub import login
import os
import time
import pandas as pd
import json
import re
from sklearn.metrics import classification_report
from qdrant_client import QdrantClient
from qdrant_client.http.models import Filter
import requests
from tqdm import tqdm
# Model related import
from pydantic import BaseModel
from openai import OpenAI
import google.generativeai as genai
from transformers import AutoTokenizer, pipeline

# Create key and secrets

In [ ]:
import os

HF_TOKEN = os.getenv("HF_TOKEN")
QDrant_KEY = os.getenv("QDrant_KEY")
Open_AI = os.getenv("OpenAI")
GOOGLE_API_KEY = os.getenv("GEMINI") 

In [4]:
from huggingface_hub import login
login(HF_TOKEN)
tokenizer = AutoTokenizer.from_pretrained("BAAI/bge-m3")

tokenizer_config.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

# Utils function

In [5]:
# --- Set SEED ---\
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
set_seed(42)
# --- Load and Prepare Data ---
def load_data(file_path):
    """Loads data and returns texts and labels."""
    data = pd.read_csv(file_path)
    texts = data["query"].tolist()
    labels = data["gpt-3.5-turbo"].tolist()
    # Remove " Question: The answer to the question..." from texts
    texts = [text.split("Question:")[0] for text in texts]
    labels = [label.split("Reasoning:")[0] for label in labels]
    return texts, labels

# --- Evaluation ---
def evaluate_results_yesno(df):
    """
    Evaluates and prints the classification report based on the DataFrame.
    Assumes the DataFrame has 'label' and 'classification' columns,
    and maps 'yes' to 1 and 'no' to 0 in the label.
    """
    if 'label' not in df.columns or 'classification' not in df.columns:
        raise ValueError("DataFrame must contain 'label' and 'classification' columns.")

    # Map labels to numerical values
    labels = df['label'].apply(lambda x: 1 if 'yes' in x.lower() else 0).tolist()
    predictions = df['classification'].apply(lambda x: 1 if 'yes' in str(x).lower() else 0).tolist()
    print(classification_report(labels, predictions))
    return classification_report(labels, predictions, output_dict=True)
# --- Evaluation ---
def evaluate_results_tsid(df):
    """
    Evaluates and prints the classification report based on the DataFrame.
    Assumes the DataFrame has 'label' and 'classification' columns,
    and maps labels to numerical values.
    """
    if 'label' not in df.columns or 'classification' not in df.columns:
        raise ValueError("DataFrame must contain 'label' and 'classification' columns.")

    # Map labels to numerical values
    label_mapping = {
        "no mental disorders": 0,
        "suicide or self-harm tendency": 1,
        "depression": 2,
        "ptsd": 3
    }
    labels = df['label'].apply(lambda x: next((i for i, label in enumerate(label_mapping) if label in x.lower()), 0)).tolist()
    predictions = df['classification'].apply(lambda x: next((i for i, label in enumerate(label_mapping) if x.lower() in label), 0)).tolist()
    # Get unique classes present in the data
    present_labels = sorted(set(labels))  # Unique label values in the dataset
    present_names = [name for name, idx in label_mapping.items() if idx in present_labels]
    print("Evaluation for Gemini API with Few-Shot:")
    report = classification_report(labels, predictions, zero_division=0,labels = present_labels,target_names =present_names, output_dict=True)
    return report
# --- Evaluation ---
def evaluate_results_sad(df):
    """
    Evaluates and prints the classification report based on the DataFrame.
    Assumes the DataFrame has 'label' and 'classification' columns,
    and maps labels to numerical values.
    """
    if 'label' not in df.columns or 'classification' not in df.columns:
        raise ValueError("DataFrame must contain 'label' and 'classification' columns.")
    # Map labels to numerical values
    label_mapping = {
        "other causes": 0,
        "school": 1,
        "financial problem": 2,
        "family issues": 3,
        "social relationships": 4,
        "work": 5,
        "health issues": 6,
        "emotional turmoil": 7,
        "everyday decision making": 8
    }
    labels = df['label'].apply(lambda x: next((i for i, label in enumerate(label_mapping) if label in x.lower()), 0)).tolist()
    predictions = df['classification'].apply(lambda x: next((i for i, label in enumerate(label_mapping) if x.lower() in label), 0)).tolist()
    # Get unique classes present in the data
    present_labels = sorted(set(labels))  # Unique label values in the dataset
    present_names = [name for name, idx in label_mapping.items() if idx in present_labels]
    print("Evaluation for Gemini API with Few-Shot:")
    report = classification_report(labels, predictions, zero_division=0,labels = present_labels,target_names =present_names, output_dict=True)
    print(report)
    return report
# --- Set Up Qdrant Client ---
def setup_qdrant_client(api_key, url="https://35612626-619b-40d3-90db-e71a27a12e38.eu-west-1-0.aws.cloud.qdrant.io:6333"):
    """Initializes Qdrant client."""
    return QdrantClient(url=url, api_key=api_key)
# --- RAG Retrieval ---
def retrieve_context(qdrant_client, collection_name, query_text, vector_size, tokenizer, top_k=10):
    """
    Retrieves relevant documents from Qdrant for a given query, with padding to ensure consistent vector size.
    """
    if len(query_text) > vector_size:
        query_text = query_text[:vector_size]
    # Generate query embedding
    query_embedding = tokenizer(query_text, return_tensors="pt", truncation=True, padding=True)["input_ids"].numpy().flatten().tolist()

    # Pad the embedding to match the required vector size
    padded_embedding = pad_embedding(query_embedding, vector_size)

    # Search for top-k similar documents
    search_result = qdrant_client.search(
        collection_name=collection_name,
        query_vector=padded_embedding,
        limit=top_k
    )
    # Extract relevant text from payloads and labels
    if len(search_result) > 0:
        # Group the similar posts based on cosine similarity (a simple check of the text since we do not have cosine similairity value)
        unique_posts = []
        unique_context = []
        for item in search_result:
             post = item.payload.get("post", "")
             label = item.payload.get("Label", "")
             reasoning = item.payload.get("Reasoning","")
             if post not in unique_posts:
                 unique_posts.append(post)
                 unique_context.append((post,label,reasoning))
        # Choose between 1-3 post from different groups.
        selected_context = random.sample(unique_context, k=min(5, len(unique_context)))
        
        return selected_context
    else:
        return []
# --- Pad Embedding ---
def pad_embedding(embedding, target_size):
    """
    Pads an embedding to the target size with zeros.

    Args:
        embedding (list of float): The original embedding.
        target_size (int): The desired dimensionality.

    Returns:
        list of float: Padded embedding.
    """
    return np.pad(embedding, (0, max(0, target_size - len(embedding))), mode='constant').tolist()
    

In [26]:
def load_model(model_type):
    model = None
    if model_type == "Gemma": 
        model = pipeline(
            "text-generation",
            model="google/gemma-2-2b-it",
            model_kwargs={"torch_dtype": torch.bfloat16},
            device="cuda")
    elif model_type == "OpenAI":
        model = OpenAI(api_key=Open_AI)
    elif model_type == "Gemini":
        genai.configure(api_key=GOOGLE_API_KEY)
        model = genai.GenerativeModel(model_name="gemini-1.5-flash")
    return model
# --- Classification with RAG ---
def classify_with_gemini(texts, labels, prompt_template, model, qdrant_client, collection_name, tokenizer,is_RAG, dataset):
    """Classifies texts using Gemini API with RAG-enhanced prompts."""
    results = []
    VECTOR_SIZE = 1024
    if dataset == "TSID":
        default_res = "no mental disorders"
    elif dataset == "SAD":
        default_res = "other causes"
    else:
        default_res = "no"
    prompt_template = prompt_template + "\n "
    for i, (text, label) in tqdm(enumerate(zip(texts, labels)), total=len(texts), desc="Processing Texts"):
        # Retrieve context using RAG
        if is_RAG:
            retrieved_examples =""
            context = retrieve_context(qdrant_client, collection_name, text, VECTOR_SIZE, tokenizer)
            for index, context in enumerate(context):
                 retrieved_examples += f"Example {index+1}:{context[0]} \nLabel for the context: {context[1]} \nReasoning for the context: {context[2]}\n"
                 prompt_template = prompt_template.replace("{retrieved_example}",retrieved_examples)
        
        # Replace the placeholders with actual data
        prompt_to_send = prompt_template.replace("{user_input}", text)

        try:
            response = model.generate_content(
                prompt_to_send,
                safety_settings={
                    "HARM_CATEGORY_HARASSMENT": "BLOCK_NONE",
                    "HARM_CATEGORY_HATE_SPEECH": "BLOCK_NONE",
                    "HARM_CATEGORY_SEXUALLY_EXPLICIT": "BLOCK_NONE",
                    "HARM_CATEGORY_DANGEROUS_CONTENT": "BLOCK_NONE",
                },
            )
            time.sleep(5)  # To respect rate limits

            # Parse the response
            response_text = response.text.strip()
            cleaned_response = re.sub(r"```json|```", "", response_text).strip()
            try:
                response_json = json.loads(cleaned_response)
                classification = response_json.get("classification", default_res)  # Default to "no" if missing
                reasoning = response_json.get("reasoning", "No specific indications of stress were found in the user's text.")
                print(f"Processed {i+1}/{len(texts)}: Classification: {classification}, Reasoning: {reasoning[:50]}...")  # Print output

            except json.JSONDecodeError as e:
                classification = default_res
                reasoning = f"Failed to parse JSON. Error: {e}"
                print(f"Processed {i+1}/{len(texts)}: Classification: {classification}, Reasoning: {reasoning}")  # Print error
            except Exception as e:
                 classification = default_res
                 reasoning = f"Error processing response: {e}"
                 print(f"Processed {i+1}/{len(texts)}: Error: {e}")  # Print error

            results.append({
                "text": text,
                "label": label,
                "classification": classification,
                "reasoning": reasoning
            })

        except Exception as e:
            results.append({
                "text": text,
                "label": label,
                "classification": "no",
                "reasoning": f"Error processing response: {e}"
            })
            print(f"Processed {i+1}/{len(texts)}: Error: {e}")  # Print error

    # Convert results to a DataFrame
    df = pd.DataFrame(results)
    return df
def classify_with_openai(texts, labels, prompt_template, pipe, qdrant_client, collection_name, tokenizer, is_RAG, dataset,quota_limit=1000):
    """Classifies texts using Groq API with RAG-enhanced prompts."""
    count = 0
    results = []
    VECTOR_SIZE = 1024
    if dataset == "TSID":
        default_res = "no mental disorders"
    elif dataset == "SAD":
        default_res = "other causes"
    else :
        default_res = "no"
    class MentalOutput(BaseModel):
        label: str
        reasoning: str
        
    for text, label in tqdm(zip(texts, labels), total=len(texts)):
        if count >= quota_limit:
            print("Quota reached. Stopping inference.")
            break
        count += 1
        # # Retrieve context using RAG
        # print(f"Using RAG {is_RAG}")
        if is_RAG:
            USER_PROMPT_TEMPLATE = """ 
            Classify the following query based on the examples provided.
            Examples:
            {examples}
            Now classify this query:
            {query}
            """
            context = retrieve_context(qdrant_client, collection_name, text, VECTOR_SIZE, tokenizer)
            example = ""
            for index, context in enumerate(context):
                 example += f"Example {index+1}:{context[0]} \nLabel for the context: {context[1]} \nReasoning for the context: {context[2]}\n"
                 USER_PROMPT_TEMPLATE = USER_PROMPT_TEMPLATE.replace("{query}",text).replace("{examples}",example)
        else:
            USER_PROMPT_TEMPLATE = """ 
            Now classify this query:
            {query}
            """
            USER_PROMPT_TEMPLATE = USER_PROMPT_TEMPLATE.replace("{query}",text)    
        print(prompt_template)
        print(USER_PROMPT_TEMPLATE)
        try:
            completion = pipe.beta.chat.completions.parse(
                model="gpt-4o-mini",
                messages=[
                    {"role": "system", "content": prompt_template},
                    {"role": "user", "content": USER_PROMPT_TEMPLATE},
                ],
                response_format=MentalOutput,
            )

            time.sleep(1)  # To respect rate limits
            event = completion.choices[0].message.parsed
            print(f"Raw Response: {event}")  # For debugging
            classification = event.label
            reasoning = event.reasoning

            results.append({
                "text": text,
                "label": label,
                "classification": classification,
                "reasoning": reasoning
            })

        except Exception as e:
            print(f"Error processing response: {e}, text: {text}")
            results.append({
                "text": text,
                "label": label,
                "classification": default_res,
                "reasoning": f"Error processing response: {e}"
            })

    # Convert results to a DataFrame
    df = pd.DataFrame(results)
    return df
def classify_with_gemma(texts, labels, prompt_template, pipe, qdrant_client, collection_name, tokenizer, is_RAG,dataset ,quota_limit=1000):
    """Classifies texts using Groq API with RAG-enhanced prompts."""
    count = 0
    results = []
    VECTOR_SIZE = 1024
    if dataset == "TSID":
        default_res = "no mental disorders"
    elif dataset == "SAD":
        default_res = "other causes"
    else :
        default_res = "no"
    for text, label in zip(texts, labels):
        if count >= quota_limit:
            print("Quota reached. Stopping inference.")
            break
        count += 1
        print(f"Processing text {count}/{len(texts)}")

        # Prepare retrieved examples for multishot prompt
        retrieved_examples = ""
        if is_RAG:
            # Retrieve context using RAG
            context = retrieve_context(qdrant_client, collection_name, text, VECTOR_SIZE, tokenizer)
            for index, context in enumerate(context):
                 retrieved_examples += f"Example {index+1}:{context[0]} \nLabel for the context: {context[1]} \nReasoning for the context: {context[2]}\n"
                 prompt_template = prompt_template.replace("{retrieved_example}",retrieved_examples)

        # Replace the placeholders with actual data
        prompt_to_send = prompt_template.replace("{user_input}", text)
        print(prompt_to_send)
        try:
            messages=[
                {
                    "role": "user",
                    "content": prompt_to_send,
                }
            ]
            outputs = pipe(messages, max_new_tokens=1024)
            
            # time.sleep(3)  # To respect rate limits

            # Parse the response
            response_text = outputs[0]["generated_text"][-1]["content"].strip()
            print(f"Raw Response: {response_text}")  # For debugging
            try:
                cleaned_response = re.sub(r"```json|```", "", response_text).strip()
                response_json = json.loads(cleaned_response)
                classification = response_json.get("classification", default_res)
                reasoning = response_json.get("reasoning", "No reasoning provided.")
            except json.JSONDecodeError as e:
                print(f"Failed to parse JSON: {e}, Response: {response_text}")
                classification = default_res
                reasoning = "Failed to parse JSON."

            results.append({
                "text": text,
                "label": label,
                "classification": classification,
                "reasoning": reasoning
            })

        except Exception as e:
            print(f"Error processing response: {e}, text: {text}")
            results.append({
                "text": text,
                "label": label,
                "classification": default_res,
                "reasoning": f"Error processing response: {e}"
            })

    # Convert results to a DataFrame
    df = pd.DataFrame(results)
    return df

In [7]:
import sys
sys.path.append("/kaggle/input/ai-therapy-prompt") 
import Prompt_model

In [18]:
def run_inference(model, dataset, method):
    isRAG = method not in ["ZS", "FS"]
    QDRANT_COLLECTION = "Mental_Llama"
    # API and Dataset Setup
    FILE_PATH = {
        "DR": "/kaggle/input/mentallamatestdata/test_data/test_complete/DR.csv",
        "SAD": "/kaggle/input/mentallamatestdata/test_data/test_complete/SAD.csv",
        "TSID": "/kaggle/input/mentallamatestdata/test_data/test_complete/t-sid.csv",
        "Dreaddit": "/kaggle/input/mentallamatestdata/test_data/test_complete/dreaddit.csv"
    }
    
    INFERENCE_FUNC = {
        "Gemma": classify_with_gemma,
        "OpenAI": classify_with_openai,
        "Gemini": classify_with_gemini
    }
    
    PROMPT_TYPE = {
        "DR": {
            "ZS": Prompt_model.ZS_Prompt_DR,
            "FS": Prompt_model.Fewshot_Prompt_DR,
            "RAG": Prompt_model.RAG_Prompt_DR,
            "Our": Prompt_model.Our_Prompt_DR
        },
        "Dreaddit": {
            "ZS": Prompt_model.ZS_Prompt_Dreaddit,
            "FS": Prompt_model.FS_Prompt_Dreaddit,
            "RAG": Prompt_model.RAG_Prompt_Dreaddit,
            "Our": Prompt_model.Our_Prompt_Dreaddit
        },
        "TSID": {
            "ZS": Prompt_model.ZS_Prompt_TSID,
            "FS": Prompt_model.Fewshot_Prompt_TSID,
            "RAG": Prompt_model.RAG_Prompt_TSID,
            "Our": Prompt_model.Our_Prompt_TSID
        },
        "SAD": {
            "ZS": Prompt_model.ZS_Prompt_SAD,
            "FS": Prompt_model.Fewshot_Prompt_SAD,
            "RAG": Prompt_model.RAG_Prompt_SAD,
            "Our": Prompt_model.Our_Prompt_SAD
        }
    }
    qdrant_client = setup_qdrant_client(QDrant_KEY)
    # Load data
    texts, labels = load_data(FILE_PATH.get(dataset))

    # Ensure the model name is correct **before** loading the model
    classify_function = INFERENCE_FUNC.get(model)
    if classify_function is None:
        raise ValueError(f"Invalid model name '{model}'. Must be one of {list(INFERENCE_FUNC.keys())}")

    # Load the model
    model_instance = load_model(model)

    # Get the correct prompt
    prompt_template = PROMPT_TYPE.get(dataset, {}).get(method)
    if prompt_template is None:
        raise ValueError(f"Invalid prompt type '{method}' for dataset '{dataset}'")

    # Classify using the correct function
    results_df = classify_function(
        texts,
        labels,
        prompt_template,
        model_instance,
        qdrant_client,
        QDRANT_COLLECTION,
        tokenizer,
        isRAG,
        dataset
    )
    # Evaluate results
    if dataset == "SAD":
        eval_report = evaluate_results_sad(results_df)
    elif dataset == "TSID":
        eval_report = evaluate_results_tsid(results_df)
    else:
        eval_report = evaluate_results_yesno(results_df)

    # Save full output with reasoning and Evaluation
    if eval_report:
        eval_df = pd.DataFrame(eval_report).transpose()
        # Fix incorrect reference to `prompt` (it should be `method`)
        eval_df.to_csv(f"{dataset}_{model}_{method}_Results.csv")  
        results_df.to_csv(f"{dataset}_{model}_{method}_DFResults.csv")  

        print(f"Evaluation and output saved successfully for {method} method.")


In [27]:
models = ["Gemma","OpenAI","Gemini"]
datasets = ["Dreaddit","DR","TSID","SAD"]
methods = ["ZS","FS","RAG","Our"]

for model in models:
    for dataset in datasets:
        for method in methods:
            print(f"Running inference for {model}_{method}_{dataset}")
            run_inference(model, dataset,method)

Running inference for Gemini_ZS_Dreaddit


Processing Texts:   0%|          | 1/414 [00:06<41:27,  6.02s/it]

Processed 1/414: Classification: yes, Reasoning: The user explicitly mentions post-traumatic stress...


Processing Texts:   0%|          | 2/414 [00:11<40:31,  5.90s/it]

Processed 2/414: Classification: yes, Reasoning: The post expresses a heightened sense of caution a...


...

Processed 3/684: Classification: no, Reasoning: The user states "i have been wanting to find anoth...


Processing Texts:   1%|          | 4/684 [00:24<1:08:34,  6.05s/it]

Processed 4/684: Classification: no, Reasoning: The post only states "The usual stressor in my lif...


Processing Texts:   1%|          | 5/684 [00:30<1:08:09,  6.02s/it]

Processed 5/684: Classification: no, Reasoning: The post mentions "stressed out" due to "bad drive...
Evaluation for Gemini API with Few-Shot:
{'financial problem': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 1.0}, 'work': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 3.0}, 'everyday decision making': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 1.0}, 'micro avg': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 5.0}, 'macro avg': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 5.0}, 'weighted avg': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 5.0}}
Evaluation and output saved successfully for Our method.
